# 01 - Crear Modelos Paramétricos

Este notebook demuestra cómo crear modelos paramétricos usando la clase `ModelBuilder`.

## Objetivos:
- Crear un modelo individual para verificar la lógica
- Generar múltiples modelos con diferentes parámetros
- Verificar la estructura de archivos generados

In [3]:
import sys
sys.path.append('../src')

from model_builder import ModelBuilder
from utils.model_helpers import ModelBuilderHelpers
from parametric_runner import ParametricRunner
import json
import os

ImportError: attempted relative import with no known parent package

## 1. Crear un Modelo Individual

In [2]:
# Inicializar el constructor de modelos y helpers
builder = ModelBuilder(output_dir="../models")
helpers = ModelBuilderHelpers(builder)

# Crear un modelo individual para verificar
L_B_ratio = 1.5  # Relación L/B
B = 10.0         # Ancho de la estructura en metros
nx = 4           # Número de ejes en X
ny = 3           # Número de ejes en Y

print(f"Creando modelo con L/B={L_B_ratio}, B={B}m, nx={nx}, ny={ny}")

# Opción 1: Usar el método core flexible
model_info = builder.create_model(L_B_ratio, B, nx, ny, model_name="ejemplo_core")
print(f"Modelo core creado: {model_info['name']}")

# Opción 2: Usar métodos de conveniencia 
model_static = helpers.create_static_only_model(L_B_ratio, B, nx, ny, "ejemplo_estatico")
print(f"Modelo estático creado: {model_static['name']}")

model_complete = helpers.create_complete_model(L_B_ratio, B, nx, ny, "ejemplo_completo", visualize=True)
print(f"Modelo completo creado: {model_complete['name']}")

print(f"\nArchivos guardados en: {builder.output_dir}/")

NameError: name 'ModelBuilderHelpers' is not defined

## 2. Verificar la Estructura del Modelo

Examinemos los detalles de uno de los modelos creados:

## 3. Demostrar Control de Visualización

La nueva arquitectura permite control granular de las visualizaciones:

In [ ]:
# Ejemplo de control granular de visualización
print("=== CONTROL DE VISUALIZACIÓN ===")

# Modelo sin visualización (análisis rápido)
model_fast = builder.create_model(
    L_B_ratio, B, nx, ny, model_name="ejemplo_rapido",
    enabled_analyses=['static', 'modal'],
    analysis_params={
        'static': {'steps': 5},
        'modal': {'num_modes': 3},
        'visualization': {'enabled': False}  # Sin visualización
    }
)
print(f"✅ Modelo rápido (sin viz): {model_fast['name']}")

# Modelo con visualización específica
model_viz = builder.create_model(
    L_B_ratio, B, nx, ny, model_name="ejemplo_con_viz",
    enabled_analyses=['static', 'modal'],
    analysis_params={
        'static': {'steps': 10},
        'modal': {'num_modes': 5},
        'visualization': {
            'enabled': True,
            'static_deformed': True,  # Deformada estática
            'modal_shapes': True,     # Formas modales
            'deform_scale': 200,
            'show_nodes': True
        }
    }
)
print(f"✅ Modelo con visualización: {model_viz['name']}")

# Usando métodos de conveniencia con visualización
model_conv = helpers.create_modal_only_model(
    L_B_ratio, B, nx, ny, "ejemplo_modal_conv",
    num_modes=8, visualize=True
)
print(f"✅ Método de conveniencia con viz: {model_conv['name']}")

## 3.1. Métodos de Conveniencia Adicionales

Los helpers incluyen varios métodos específicos para diferentes tipos de análisis:

In [ ]:
print("=== MÉTODOS DE CONVENIENCIA ADICIONALES ===")

# Modelo dinámico (estático + dinámico)
model_dynamic = helpers.create_dynamic_model(
    L_B_ratio, B, nx, ny, "ejemplo_dinamico",
    dt=0.005, num_steps=2000, visualize=True
)
print(f"✅ Modelo dinámico: {model_dynamic['name']}")

# Modelo para investigación (todos los análisis)
model_research = helpers.create_research_model(
    L_B_ratio, B, nx, ny, "ejemplo_investigacion",
    static_steps=20, num_modes=10, dt=0.001, num_steps=5000, visualize=True
)
print(f"✅ Modelo de investigación: {model_research['name']}")

# Modelo de prueba rápida (parámetros mínimos)
model_quick = helpers.create_quick_test_model(
    L_B_ratio, B, nx, ny, "ejemplo_prueba_rapida"
)
print(f"✅ Modelo de prueba rápida: {model_quick['name']}")

print(f"\n📊 Total de modelos creados hasta ahora:")
model_files = [f for f in os.listdir("../models") if f.endswith('.json')]
print(f"   Archivos JSON: {len(model_files)}")

In [ ]:
# Cargar y mostrar información del primer modelo creado (modelo core)
with open(model_info['file_path'], 'r') as f:
    model_data = json.load(f)

print("=== INFORMACIÓN DEL MODELO CORE ===")
print(f"Nombre: {model_data['name']}")
print(f"Parámetros:")
for key, value in model_data['parameters'].items():
    print(f"  {key}: {value}")

print(f"\nGeometría:")
print(f"  Dimensiones: {model_data['geometry']['length']} x {model_data['geometry']['width']} m")
print(f"  Ejes: {model_data['geometry']['nx']} x {model_data['geometry']['ny']}")
print(f"  Número de nodos: {len(model_data['nodes'])}")
print(f"  Número de elementos: {len(model_data['elements'])}")

print(f"\nConfiguración de Análisis:")
if 'analysis_config' in model_data:
    config = model_data['analysis_config']
    print(f"  Análisis habilitados: {config.get('enabled_analyses', ['static', 'modal'])}")
    
    if 'static' in config:
        print(f"  Estático - Pasos: {config['static'].get('steps', 10)}")
    if 'modal' in config:
        print(f"  Modal - Modos: {config['modal'].get('num_modes', 6)}")
    if 'dynamic' in config:
        print(f"  Dinámico - dt: {config['dynamic'].get('dt', 0.01)}, Pasos: {config['dynamic'].get('num_steps', 1000)}")
        
    if 'visualization' in config:
        viz = config['visualization']
        print(f"  Visualización habilitada: {viz.get('enabled', True)}")
        if viz.get('enabled', True):
            print(f"    - Deformada estática: {viz.get('static_deformed', True)}")
            print(f"    - Formas modales: {viz.get('modal_shapes', True)}")
else:
    print("  Configuración de análisis por defecto (estático + modal)")

print(f"\nArchivo guardado en: {model_data['file_path']}")

## 4. Verificar Nombres de Modelos

In [ ]:
# Probar diferentes combinaciones de parámetros
test_cases = [
    (1.0, 5.0, 2, 2),   # L/B=1.0, B=5m, nx=2, ny=2
    (1.5, 10.0, 4, 3),  # L/B=1.5, B=10m, nx=4, ny=3
    (2.0, 15.0, 6, 4),  # L/B=2.0, B=15m, nx=6, ny=4
    (2.5, 20.0, 8, 5),  # L/B=2.5, B=20m, nx=8, ny=5
]

print("=== NOMBRES DE MODELOS GENERADOS ===")
for L_B, B, nx, ny in test_cases:
    name = builder.generate_model_name(L_B, B, nx, ny)
    print(f"L/B={L_B}, B={B}m, nx={nx}, ny={ny} -> {name}")

## 5. Generar Múltiples Modelos

In [ ]:
# Definir rangos de parámetros para análisis paramétrico
L_B_ratios = [1.0, 1.5, 2.0]
B_values = [5.0, 10.0, 15.0]
nx_values = [3, 4]
ny_values = [2, 3]

print("Generando modelos paramétricos...")
print(f"Relaciones L/B: {L_B_ratios}")
print(f"Anchos B: {B_values}")
print(f"Valores de nx: {nx_values}")
print(f"Valores de ny: {ny_values}")
print(f"Total de combinaciones: {len(L_B_ratios) * len(B_values) * len(nx_values) * len(ny_values)}")

# Crear ParametricRunner para generar múltiples modelos
runner = ParametricRunner(
    L_B_ratios=L_B_ratios,
    B_values=B_values,
    nx_values=nx_values,
    ny_values=ny_values,
    models_dir="../models",
    results_dir="../results"
)

# Generar todos los modelos
models = runner.generate_parametric_models(L_B_ratios, B_values, nx_values, ny_values)

print(f"\nModelos creados exitosamente: {len(models)}")

## 6. Verificar Archivos Generados

In [ ]:
# Listar archivos generados
model_files = os.listdir("../models")
json_files = [f for f in model_files if f.endswith('.json')]

print(f"Archivos JSON en el directorio models: {len(json_files)}")
print("\nPrimeros 10 archivos:")
for i, file in enumerate(sorted(json_files)[:10]):
    print(f"{i+1}. {file}")

## 7. Verificar Dimensiones y Nombres

In [ ]:
# Verificar que las dimensiones se calculan correctamente
print("=== VERIFICACIÓN DE DIMENSIONES ===")
for L_B_ratio in L_B_ratios:
    for B in B_values:
        # Calcular dimensiones usando la lógica del ModelBuilder
        L = L_B_ratio * B
        print(f"L/B={L_B_ratio}, B={B}m -> L={L:.1f}m, B={B:.1f}m")
        print(f"  Verificación: L/B = {L/B:.1f}")

# Verificar nombres de modelo
print("\n=== VERIFICACIÓN DE NOMBRES ===")
for L_B_ratio in [1.0, 1.5]:
    for B in [10.0, 15.0]:
        for nx in [3, 4]:
            for ny in [2, 3]:
                name = builder.generate_model_name(L_B_ratio, B, nx, ny)
                print(f"L/B={L_B_ratio}, B={B}m, nx={nx}, ny={ny} -> {name}")

## 8. Resumen de Verificación

In [ ]:
print("=== RESUMEN DE VERIFICACIÓN ===")
print(f"✓ Modelos creados: {len(models)}")
print(f"✓ Archivos generados: {len(json_files)}")
print(f"✓ Directorio de salida: {builder.output_dir}")
print(f"✓ Parámetros fijos configurados: {len(builder.fixed_params)}")

# Verificar que al menos un modelo se creó correctamente
if models:
    sample_model = models[0]
    print(f"\n✓ Ejemplo de modelo creado:")
    print(f"  - Nombre: {sample_model['name']}")
    print(f"  - Parámetros: L/B={sample_model['parameters']['L_B_ratio']}, ")
    print(f"    B={sample_model['parameters']['B']}m, ")
    print(f"    nx={sample_model['parameters']['nx']}, ny={sample_model['parameters']['ny']}")
    print(f"  - Nodos: {len(sample_model['nodes'])}")
    print(f"  - Elementos: {len(sample_model['elements'])}")
    print(f"  - Archivo: {sample_model['file_path']}")
    
    print("\n✅ Verificación completada exitosamente!")